In [32]:
import numpy as np

The random number generator.

In [34]:
class RNG:
    def __init__(self, bin_size: int) -> None:
        self.bin_size = bin_size
        self.pointer = 0
        self.numbers = np.random.rand(self.bin_size)

    def _refresh(self, bin_size: int) -> None:
        self.bin_size = bin_size
        self.pointer = 0
        self.numbers = np.random.rand(self.bin_size)
    
    def _next(self) -> None:
        X = self.numbers[self.pointer]
        self.pointer += 1
        if self.pointer >= self.bin_size:
            self._refresh(self.bin_size)
        return X

    def get_random(self, max_size: int) -> int:
        return int(np.floor(max_size * self._next()))

Each community (including the background graph) could then get its own random number generator, which would be initalized once we know how many loops and multi-edges there are.

In [46]:
class AbstractCommunity(AbstractGraph):
    def __init__(self, edges: list[Edge], community_id: int) -> None:
        self.community_id = community_id
        self._adj_dict: dict[Edge, int] = {}
        self._bad_edges: list[Edge] = []

        self._diagnostics = {
            "num_loops": 0,
            "num_multi_edges": 0,
        }

        for edge in edges:
            if edge.is_loop:
                self._bad_edges.append(edge)
                self._diagnostics["num_loops"] += 1

            if edge in self.adj_dict:
                self._bad_edges.append(edge)
                self._adj_dict[edge] += 1
                self._diagnostics["num_multi_edges"] += 1
            else:
                self._adj_dict[edge] = 1

        self._edges = edges
        
        ########################################
        num_bad = self._diagnostics["num_loops"] + self._diagnostics["num_multi_edges"]
        bin_size = num_bad * (num_bad + 1) / 2 #This is the maximum possible number of rewirings 
        self._rng = RNG(bin_size)
        ########################################

NameError: name 'AbstractGraph' is not defined

Now, during rewiring, we call from self._rng

In [47]:
 def rewire_community(self) -> None:
    while len(self._bad_edges) > 0:
        for edge in self._bad_edges:
            ##########################################
            other_edge = choose_other_edge(self, edge)
            ##########################################
            rewire_edge(self.adj_dict, edge, other_edge)

        new_bad_edges = build_recycle_list(self.adj_dict)
        if len(new_bad_edges) >= len(self._bad_edges):
            self.push_to_background(new_bad_edges, self._deg_b)
            return
        else:
            self._bad_edges = new_bad_edges

def choose_other_edge(community: "Community", edge: "Edge") -> "Edge":
    choices = list(community.adj_dict.keys())
    other_edge = choices[community._rng.get_random(len(choices))]
    while other_edge == edge:
        other_edge = choices[community._rng.get_random(len(choices))]

    return other_edge

Same deal for the background graph.
This time, "self" isn't a community though. Not sure how to deal with this.

In [ ]:
def rewire_graph(self) -> "ABCDGraph":
    bad_edges = build_recycle_list(self._adj_dict)

    while len(bad_edges) > 0:
        for edge in bad_edges:
            other_edge = choose_other_edge(self, edge)
            rewire_edge(self._adj_dict, edge, other_edge)

        bad_edges = build_recycle_list(self._adj_dict)

    return self